# 工具  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
工具让代理可以在真实世界中“行动”。 
精心的描述可以帮助代理学会如何使用你的工具。

LangChain 支持多种工具格式和工具集。这里涵盖一些常见场景，更多信息请查看[文档](https://docs.langchain.com/oss/python/langchain/tools)。

## 设置

加载并/或检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env("example.env")


未找到文件 example.env。
此脚本用于再次核对 notebook 的关键配置。
这只是检查步骤，并非必需。



## 计算器示例

在这个示例中，LLM 会利用文档字符串以及推断出的参数和类型来判断何时、如何调用该工具。

In [2]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """对两个实数执行基础算术运算。"""
    print("🧮 调用计算器工具")
    # 执行指定的运算
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("不允许被零除。")
        return a / b
    else:
        raise ValueError(f"无效的操作: {operation}")

In [3]:
from langchain.agents import create_agent
from langchain_qwq import ChatQwen
import os
llm=ChatQwen(
    model="qwen3-max",
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY")
)
agent = create_agent(
    model=llm,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

这会调用你的计算器工具。

In [4]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "what is 3.1125 * 4.1234"}]}
)
print(result["messages"][-1].content)

🧮 调用计算器工具
The product of $ 3.1125 \times 4.1234 $ is $ 12.8340825 $.


我们可以在 [LangSmith Observability](https://smith.langchain.com/public/b77bde6c-f0ad-4256-bfab-7d514ece3405/r) 中查看**元数据**来确认这一点。

工具描述会产生很大影响。 
因为输入是整数，这次可能不会调用你的计算器工具。（每次运行结果可能不同）

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

🧮 调用计算器工具
The result of $ 3 \times 4 $ is $ 12.0 $.


尽管输入是实数，这种情况下通常不会调用该工具。（每次运行结果可能不同）

In [6]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

🧮 调用计算器工具
The result of $ 3.0 \times 4.0 $ is $ 12.0 $.


## 添加更详细的描述
尽管基本描述通常已足够，LangChain 也支持更丰富的描述。下面的示例使用了一种方式：Google 风格的参数描述。配合 `parse_docstring=True` 使用时，会解析并将参数描述传递给模型。你可以重命名工具并修改其描述。当你共享一个标准工具但希望加入面向代理的指令时，这会很有效。

In [8]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "对两个实数执行基础算术运算。"
        "当你需要对任何数字进行运算时都可以使用，即便是整数。"
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """对两个实数执行基础算术运算。

    Args:
        a (float): 第一个操作数。
        b (float): 第二个操作数。
        operation (Literal['add','subtract','multiply','divide']): 要执行的操作。

    Returns:
        float: 计算结果。

    Raises:
        ValueError: 当操作无效或尝试被零除时抛出。
    """
    print("🧮  调用计算器工具")
    # 执行指定的运算
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("不允许被零除。")
        return a / b
    else:
        raise ValueError(f"无效的操作: {operation}")


In [9]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant",
)

In [10]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3.0 * 4.0"}]})
print(result["messages"][-1].content)

🧮  调用计算器工具
The result of $ 3.0 \times 4.0 $ is $ 12.0 $.


让我们查看 [LangSmith Observability trace](https://smith.langchain.com/public/7d65902c-bd3c-4fc6-bbd3-7c1d66566fda/r)，看看工具描述。

In [10]:
result = agent.invoke({"messages": [{"role": "user", "content": "what is 3 * 4"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
12


## 试一试。
创建你自己的工具并在这里尝试！

In [11]:

@tool
def your_tool(
    a: float, b: float, 
) -> float:
    """Perform your favorite operation

    Args:
        a (float): operator a description
        b (float): operator b description

    Returns:
        float: description
    """
    pass

In [12]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[your_tool],
    system_prompt="You are a helpful assistant",
)